# 생물정보학 및 실습 1 - Term Project (Free Analysis)
생물정보학 및 실습 1   
서울대학교 협동과정 생물정보학전공 2022년 1학기

In [1]:
from collections import Counter, defaultdict
import math
import os
import sys
import pickle
import time

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import pysam

In [2]:
CLIP_BAM_PATH = '../data/binfo1-datapack1/CLIP-35L33G.bam'
FASTA_DIR_PATH    = '../data/chromFa'

## 1. Prepare Reference FASTAs
+ `mm39.chromFa.tar.gz` was downloaded from UCSC Genome Browser
+ FASTAs were extracted into `FASTA_DIR_PATH` by `tar -zxvf`

In [3]:
# Make dctFastaMatch that has {gencode chromID : RefSeq chromID}
lstRefSeqFa = os.listdir(FASTA_DIR_PATH)
dctFastaMatch = {}
for fa in lstRefSeqFa:
    chrId = fa.replace('.fa', '')
    if '_' not in fa:
        dctFastaMatch[chrId] = fa
    else:
        dctFastaMatch[chrId.split('_')[1].replace('v', '.')] = fa

In [4]:
# Validation
dctFastaMatch['GL456210.1']

'chr1_GL456210v1_random.fa'

In [5]:
del dctFastaMatch['chrM']

## 2. Collect Putative Binding Sequences

In [6]:
def get_chr_binding_positions(chrId):
    pileUp = pysam.AlignmentFile(CLIP_BAM_PATH).pileup(chrId)
    lstBindingPositionsPos, lstBindingPositionsNeg = [], []
    for col in pileUp:
        bases = col.get_query_sequences()
        basesPos = [base for base in bases if base.isupper()]
        basesNeg = [base for base in bases if base.islower()]
        if len(basesPos) >= 50:
            cres = stats.entropy(list(Counter(bases).values()), base=2)
            if cres >= 0.8:
                lstBindingPositionsPos.append(col.reference_pos)
        if len(basesNeg) >= 50:
            cres = stats.entropy(list(Counter(bases).values()), base=2)
            if cres >= 0.8:
                lstBindingPositionsNeg.append(col.reference_pos)
    return lstBindingPositionsPos, lstBindingPositionsNeg

In [7]:
# Complement: A <--> T, G <--> C
# Dictionary for the Unicode code point
dctComplement = str.maketrans('ACGT', 'TGCA')

In [8]:
def get_binding_sequences(chrId):
    lstBindingPositionsPos, lstBindingPositionsNeg = get_chr_binding_positions(chrId)

    FASTA_FILE_PATH = os.path.join(FASTA_DIR_PATH, dctFastaMatch[chrId])
    with open(FASTA_FILE_PATH, 'rt') as fIn:
        assert next(fIn).startswith('>') # Skip header
        seq = fIn.read().strip().replace('\n', '').upper() # Remove new line chr and make lowercase be uppercase

    dctSeqs = Counter()
    for pos in lstBindingPositionsPos:
        dctSeqs.update([seq[pos-2:pos+4]])
    for pos in lstBindingPositionsNeg:
        hexamer = seq[pos-3:pos+3][::-1].translate(dctComplement)
        dctSeqs.update([hexamer])
    return dctSeqs

In [9]:
dctSeqs = Counter()
for chrId, fa in dctFastaMatch.items():
    print(time.ctime(), f'{fa} now opens', sep=' --- ')
    dctSeqs.update(get_binding_sequences(chrId))

Fri Jun  3 12:53:19 2022 --- chr10.fa now opens
Fri Jun  3 12:55:34 2022 --- chr11.fa now opens
Fri Jun  3 12:58:55 2022 --- chr12.fa now opens
Fri Jun  3 13:00:48 2022 --- chr13.fa now opens
Fri Jun  3 13:02:49 2022 --- chr14.fa now opens
Fri Jun  3 13:04:36 2022 --- chr15.fa now opens
Fri Jun  3 13:06:28 2022 --- chr16.fa now opens
Fri Jun  3 13:07:58 2022 --- chr17.fa now opens
Fri Jun  3 13:09:51 2022 --- chr18.fa now opens
Fri Jun  3 13:11:09 2022 --- chr19.fa now opens
Fri Jun  3 13:12:15 2022 --- chr1.fa now opens
Fri Jun  3 13:16:29 2022 --- chr1_GL456210v1_random.fa now opens
Fri Jun  3 13:16:29 2022 --- chr1_GL456211v1_random.fa now opens
Fri Jun  3 13:16:30 2022 --- chr1_GL456212v1_random.fa now opens
Fri Jun  3 13:16:30 2022 --- chr1_GL456221v1_random.fa now opens
Fri Jun  3 13:16:30 2022 --- chr1_GL456239v1_random.fa now opens
Fri Jun  3 13:16:30 2022 --- chr1_MU069434v1_random.fa now opens
Fri Jun  3 13:16:30 2022 --- chr2.fa now opens
Fri Jun  3 13:19:58 2022 --- chr3.fa

In [10]:
dctSeqs.most_common(20)

[('AAGGAG', 1108),
 ('AAGAAG', 988),
 ('AAGAGA', 833),
 ('GAGGAG', 824),
 ('TGGAGA', 680),
 ('AAGAGG', 680),
 ('AGGAGA', 611),
 ('GAGAAG', 607),
 ('GAGAGA', 571),
 ('AAGGTG', 571),
 ('AAGATG', 563),
 ('AGGAAG', 503),
 ('AGGAGG', 476),
 ('AAGAAA', 454),
 ('CAGAGA', 452),
 ('ATGGAG', 438),
 ('CTGGAG', 426),
 ('ATGAGA', 412),
 ('CAGAGG', 408),
 ('AAGAGT', 395)]

In [11]:
dfHexamers = pd.DataFrame(dctSeqs.most_common(), columns=['hexamer', 'counts']).set_index('hexamer')
dfHexamers.head()

,counts
hexamer,
AAGGAG,1108
AAGAAG,988
AAGAGA,833
GAGGAG,824
TGGAGA,680


In [12]:
dfHexamers.to_csv('../stats/hexamers.txt', sep='\t')